In [11]:
from datasets import load_dataset

from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM
dataset = load_dataset('json', data_files='/home/ghan/datasets/diabetes.json')
dataset_length = len(dataset['train'])
indices = list(range(dataset_length))
train_indices, test_indices = train_test_split(indices, test_size=0.2, random_state=42)
train_dataset = dataset['train'].select(train_indices)
test_dataset = dataset['train'].select(test_indices)

Using custom data configuration default-ee7cb05032dae5f7
Found cached dataset json (/home/ghan/.cache/huggingface/datasets/json/default-ee7cb05032dae5f7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
from datasets import ClassLabel
import torch

In [13]:
test_dataset

Dataset({
    features: ['did', 'uid', 'text', 'gender', 'age', 'date', 'label'],
    num_rows: 253
})

In [70]:
# 初始化一个空的 set
label_set = set()

# 遍历数据集
for example in train_dataset:
    # 添加所有的标签到 set 中
    label_set.update(example["label"])
for example in test_dataset:
    # 添加所有的标签到 set 中
    label_set.update(example["label"])

# 将 set 转换为 list
label_list = list(label_set)

In [71]:
label_list

['mi-6mos',
 'major-diabetes',
 'keto-1yr',
 'creatinine',
 'hba1c',
 'asp-for-mi',
 'alcohol-abuse',
 'dietsupp-2mos',
 'drug-abuse',
 'abdominal']

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['did', 'uid', 'text', 'gender', 'age', 'date', 'label'],
        num_rows: 1265
    })
})

In [15]:
from transformers import BertTokenizer
from datasets import DatasetDict

# Instantiate a tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [16]:
def encode(examples):
    encoded = tokenizer(examples['text'])
    examples['length'] = len(encoded['input_ids'])
    return examples

In [17]:
dataset = dataset.map(encode)

Loading cached processed dataset at /home/ghan/.cache/huggingface/datasets/json/default-ee7cb05032dae5f7/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-a4ac659b453760cb.arrow


In [7]:
length = dataset['train']['length'][0]


In [8]:
length

407

In [18]:
lengths = dataset['train']['length']

In [19]:
lengths

[407,
 336,
 419,
 292,
 338,
 557,
 593,
 987,
 634,
 1386,
 495,
 1731,
 361,
 601,
 1393,
 1048,
 711,
 341,
 576,
 1325,
 435,
 431,
 598,
 1168,
 2014,
 1046,
 526,
 1281,
 307,
 423,
 705,
 471,
 535,
 428,
 383,
 844,
 1938,
 889,
 1059,
 734,
 863,
 171,
 439,
 187,
 383,
 1137,
 1544,
 1153,
 609,
 1400,
 637,
 345,
 1045,
 881,
 347,
 329,
 1510,
 1320,
 393,
 321,
 705,
 779,
 948,
 1475,
 282,
 1251,
 825,
 522,
 1036,
 565,
 324,
 475,
 826,
 1208,
 490,
 464,
 404,
 397,
 417,
 309,
 383,
 1206,
 1668,
 356,
 586,
 741,
 1334,
 917,
 807,
 844,
 789,
 611,
 1252,
 484,
 610,
 1398,
 2105,
 3106,
 1446,
 242,
 350,
 482,
 495,
 1366,
 627,
 350,
 430,
 1822,
 1595,
 384,
 220,
 606,
 550,
 1354,
 1126,
 811,
 1465,
 519,
 721,
 960,
 1347,
 474,
 1151,
 195,
 1050,
 550,
 288,
 391,
 532,
 322,
 611,
 317,
 348,
 599,
 447,
 687,
 503,
 369,
 653,
 630,
 698,
 401,
 585,
 506,
 1755,
 666,
 382,
 239,
 514,
 1658,
 428,
 249,
 136,
 1030,
 1154,
 301,
 823,
 690,
 1487,
 5

In [20]:
def count_elements(arr):
    count_512 = 0
    count_1024 = 0
    count_2048 = 0
    count_above_2048 = 0

    for num in arr:
        if num <= 512:
            count_512 += 1
        elif num <= 1024:
            count_1024 += 1
        elif num <= 2048:
            count_2048 += 1
        else:
            count_above_2048 += 1

    total = len(arr)
    ratio_512 = count_512 / total
    ratio_1024 = count_1024 / total
    ratio_2048 = count_2048 / total
    ratio_above_2048 = count_above_2048 / total

    return count_512, ratio_512, count_1024, ratio_1024, count_2048, ratio_2048, count_above_2048, ratio_above_2048

In [22]:
count_elements(lengths)

(502,
 0.3968379446640316,
 489,
 0.3865612648221344,
 263,
 0.20790513833992094,
 11,
 0.008695652173913044)

In [23]:
from sklearn.model_selection import train_test_split


In [24]:
dataset_length = len(dataset['train'])
indices = list(range(dataset_length))
train_indices, temp_test_indices = train_test_split(indices, test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_test_indices, test_size=0.5, random_state=42)
train_dataset = dataset['train'].select(train_indices)
val_dataset=dataset['train'].select(val_indices)
test_dataset = dataset['train'].select(test_indices)

In [25]:
test_lengths=test_dataset['length']

In [26]:
test_lengths

[255,
 816,
 824,
 637,
 799,
 606,
 323,
 435,
 1315,
 1064,
 447,
 333,
 1033,
 348,
 1334,
 519,
 193,
 353,
 664,
 373,
 936,
 450,
 480,
 440,
 1400,
 718,
 302,
 635,
 527,
 722,
 945,
 476,
 449,
 614,
 1000,
 399,
 751,
 347,
 1123,
 557,
 2131,
 630,
 227,
 282,
 1251,
 422,
 498,
 931,
 842,
 1510,
 1218,
 1327,
 963,
 138,
 411,
 277,
 684,
 1475,
 1127,
 1195,
 126,
 429,
 423,
 512,
 734,
 393,
 1059,
 700,
 447,
 401,
 380,
 334,
 516,
 1487,
 389,
 1271,
 627,
 687,
 566,
 1574,
 239,
 570,
 235,
 503,
 582,
 716,
 236,
 2105,
 351,
 1206,
 576,
 823,
 836,
 810,
 724,
 1113,
 588,
 164,
 620,
 516,
 889,
 182,
 292,
 464,
 409,
 275,
 417,
 830,
 256,
 1067,
 682,
 705,
 1067,
 1108,
 361,
 519,
 504,
 1469,
 821,
 1024,
 345,
 1651,
 671,
 726,
 464,
 780,
 992,
 140,
 1248,
 1603,
 404,
 336,
 412,
 321,
 212,
 1064,
 894,
 471,
 606,
 1168,
 442,
 1165,
 466,
 479,
 129,
 266,
 456,
 312,
 729,
 920,
 1283,
 1264,
 594,
 387,
 562,
 912,
 654,
 689,
 508,
 947,
 428,

In [27]:
count_elements(test_lengths)

(84,
 0.4421052631578947,
 68,
 0.35789473684210527,
 36,
 0.18947368421052632,
 2,
 0.010526315789473684)

In [28]:
quartiles=[512,1024,2048]

In [29]:
dataset

DatasetDict({
    train: Dataset({
        features: ['did', 'uid', 'text', 'gender', 'age', 'date', 'label', 'length'],
        num_rows: 1265
    })
})

In [30]:
def add_length_feature(examples):

    length=examples['length']
    if length <= quartiles[0]:
        length_feature=1
    elif length <= quartiles[1]:
        length_feature=2
    elif length <= quartiles[2]:
        length_feature=3
    else:
        length_feature=3
    examples['length_feature'] = length_feature
    return examples

dataset = dataset.map(add_length_feature)

  0%|          | 0/1265 [00:00<?, ?ex/s]

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['did', 'uid', 'text', 'gender', 'age', 'date', 'label', 'length', 'length_feature'],
        num_rows: 1265
    })
})

In [32]:
# Convert the 'train' dataset to a Pandas DataFrame
train_df = dataset['train'].to_pandas()

# Save the DataFrame as a JSON file
train_df.to_json("/home/ghan/datasets/diabetes3.json", orient="records", lines=True)



In [33]:
dataset2 = load_dataset('json', data_files='/home/ghan/datasets/diabetes3.json')

Using custom data configuration default-eee93eb64458fe47


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/ghan/.cache/huggingface/datasets/json/default-eee93eb64458fe47/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['did', 'uid', 'text', 'gender', 'age', 'date', 'label', 'length', 'length_feature'],
        num_rows: 1265
    })
})

In [36]:
dataset2['train']['length_feature']

[1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 1,
 3,
 1,
 2,
 3,
 3,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 3,
 3,
 3,
 2,
 3,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 3,
 2,
 3,
 2,
 2,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 2,
 3,
 2,
 1,
 3,
 2,
 1,
 1,
 3,
 3,
 1,
 1,
 2,
 2,
 2,
 3,
 1,
 3,
 2,
 2,
 3,
 2,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 1,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 3,
 1,
 2,
 3,
 3,
 3,
 3,
 1,
 1,
 1,
 1,
 3,
 2,
 1,
 1,
 3,
 3,
 1,
 1,
 2,
 2,
 3,
 3,
 2,
 3,
 2,
 2,
 2,
 3,
 1,
 3,
 1,
 3,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 3,
 2,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 3,
 3,
 1,
 2,
 2,
 3,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 3,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 3,
 3,
 1,
 3,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 3,
 3,
 3,
 2,
 3,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 3,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,


In [8]:
# 遍历数据集
for i, example in enumerate(dataset['train']):
    # 检查 "label" 字段是否是列表
    if not isinstance(example["label"], list):
        print(f"Error: example {i}'s label is not a list. It's a {type(example['label'])}.")
    else:
        # 检查列表中的每一个元素是否都是字符串
        for j, label in enumerate(example["label"]):
            if not isinstance(label, str):
                print(f"Error: example {i}'s label at index {j} is not a string. It's a {type(label)}.")


In [9]:
num_labels = len(label_list)
label_encoder = ClassLabel(num_classes=len(label_list), names=label_list)
label_list_encoded = list(range(len(label_list)))

In [28]:
import json

# Convert the datasets to dictionaries
json_data = {split: dataset[split].to_dict() for split in dataset.keys()}

# Save the dictionaries as a JSON file
with open("/home/ghan/datasets/diabetes2.json", "w") as json_file:
    json.dump(json_data, json_file)

In [10]:
def preprocess_function(examples):


    text=examples['text']

    batch = tokenizer(text, max_length=512, truncation=True)


    labels_encoded = [[label_encoder.str2int(label) for label in labels] for labels in examples["label"]]
    batch["labels"] = torch.tensor([[1 if label in labels else 0 for label in label_list_encoded] for labels in labels_encoded])
    return batch

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [12]:

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
                desc="Running tokenizer on train dataset",
            )

Running tokenizer on train dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

In [13]:
train_dataset

Dataset({
    features: ['did', 'uid', 'text', 'gender', 'age', 'date', 'label', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1012
})

In [51]:
first=train_dataset[2]

In [52]:
first

{'did': '156',
 'uid': '179',
 'text': 'medicine junior admit note choi date of admission january 29 2070 cardiologist calvin null cc 61 yo male former smoker hyperlipidemia with increasing frequency of chest pain over the past days 61 yo male crf former smoker htn obesity present with days of increasing chest pain patient reports that he began having pain at the sternal border no radiation days ago patient unsure when pain initially began patient describes suprasternal pain sharp pain that felt like gas pain initially lasting for minutes pain came and went without any correlation with activity episodes increased in frequency and intensity over the days exercise did not worsen his pain and was able to climb flights of stairs without exacerbation of symptoms pain has been increasing in intensity over the past days pain became unbearable and patient finally presented to bra pt did not have any clear ste given iv nitro heparin and asa evaluated by bra cardiologist and transferred to cch c

In [53]:
if "label" in first and first["label"] is not None:
    print('yes')

yes
